In [ ]:
import re
import requests
import time
import pandas as pd
import json

In [ ]:
final_result = pd.DataFrame(
    columns=['village_code', 'province_name', 'city_name', 'district_name', 'town_name', 'village_name'])

failed_list = []

url = 'http://www.stats.gov.cn/tjsj/tjbz/tjyqhdmhcxhfdm/2019/'
province_response = requests.get(url)
province_response.raise_for_status()
province_response.encoding = province_response.apparent_encoding
province_response.close()
pattern = re.compile("<a href='(.*?)'>(.*?)<")

In [ ]:
# # 爬取所有省的url
province_result = list(set(re.findall(pattern, province_response.text)))
for province in [('22.html', '吉林省')]:
    temp_province_name = province[1]
    print(temp_province_name + ': start')
    temp_province_url = url + province[0]
    try:
        city_response = requests.get(temp_province_url)
        city_response.raise_for_status()
        city_response.encoding = city_response.apparent_encoding
        city_response.close()
        # 爬取省下面所有市的url
        city_result_1 = list(set(re.findall(pattern, city_response.text)))
        city_result = []
        for r in city_result_1:
            if '0' not in r[1]:
                city_result.append(r)
        print(city_result)
        for city in city_result:
            temp_city_name = city[1]

            temp_city_url = url + city[0]
            try:
                district_response = requests.get(temp_city_url)
                district_response.raise_for_status()
                district_response.encoding = district_response.apparent_encoding
                district_response.close()
                # 爬取城市下所有区县的url
                district_result_1 = list(set(re.findall(pattern, district_response.text)))
                district_result = []
                for r in district_result_1:
                    if '0' not in r[1]:
                        district_result.append(r)
                for district in district_result:
                    temp_district_name = district[1]
                    temp_district_url = url + district[0][3:5] + '/' + district[0]
                    try:
                        town_response = requests.get(temp_district_url)
                        town_response.raise_for_status()
                        town_response.encoding = town_response.apparent_encoding
                        town_response.close()
                        # pattern4 = re.compile("<tr class='villagetr'><td>(.*?)</td><td>.*?</td><td>(.*?)</td></tr>")
                        town_result_1 = list(set(re.findall(pattern, town_response.text)))
                        town_result = []
                        for r in town_result_1:
                            if '0' not in r[1]:
                                town_result.append(r)
                        for town in town_result:
                            temp_town_name = town[1]
                            temp_town_url = url + town[0][3:5] + '/' + town[0][5:7] + '/' + town[0]
                            try:
                                village_response = requests.get(temp_town_url)
                                village_response.raise_for_status()
                                village_response.encoding = village_response.apparent_encoding
                                village_response.close()
                                pattern4 = re.compile(
                                    "<tr class='villagetr'><td>(.*?)</td><td>.*?</td><td>(.*?)</td></tr>")
                                village_result = list(set(re.findall(pattern4, village_response.text)))
                                for village in village_result:
                                    village_code = village[0]
                                    village_name = village[1]
                                    final_result = final_result.append(
                                        {'village_code': village_code, 'province_name': temp_province_name,
                                         'city_name': temp_city_name,
                                         'district_name': temp_district_name, 'town_name': temp_town_name,
                                         'village_name': village_name}, ignore_index=True)
                            except:
                                failed_list.append({'province_name': temp_province_name,
                                                    'city_name': temp_city_name,
                                                    'district_name': temp_district_name, 'town_name': temp_town_name,
                                                    'town_url': temp_town_url, 'level': 4})
                    except:
                        failed_list.append({'province_name': temp_province_name,
                                            'city_name': temp_city_name,
                                            'district_name': temp_district_name,
                                            'district_url': temp_district_url, 'level': 3})
            except:
                failed_list.append({'province_name': temp_province_name,
                                    'city_name': temp_city_name,
                                    'city_url': temp_city_url, 'level': 2})
    except:
        failed_list.append({'province_name': temp_province_name,
                            'province_url': temp_province_url, 'level': 1})


In [ ]:
final_result.to_excel('final_result_hn.xlsx')
print(final_result.shape[0])

with open('hn.txt', 'w') as file:
    file.write(json.dumps(failed_list))

print('well_done')
